# A1 - Proposta de Modelo de Síntese de Dado Tabular em Saúde

#### Aluno: Tiago Amorim (100.675)

Esta atividade pode ser realizada individualmente ou em duplas (em caso de entrega em dupla, pede-se que apenas um dos membros da dupla realize a entrega e indique isso explicitamente no campo de comentários da entrega).
Atente-se ao horário de entrega da atividade!

O objetivo é que você proponha e gere um banco de dados sintéticos de resultados de exames de hemograma.
Não precisa ser uma abordagem baseada em deep learning, pode ser uma abordagem baseada em regras ou em modelos estatísticos clássicos.

Seguem anexadas três planilhas extraídas de uma base de resultados de exames de laboratório de análises clínicas.
São dados reais obtidos de um hemocentro do estado de São Paulo (e podem exigir etapas de limpeza de dados).
Campos de identificação foram transformads para fins de anonimização completa dos pacientes.

Proponha um método, implemente e apresente uma base de dados sintética com, no mínimo, 1000 resultados de exames fictícios.

Simplificação admitida: sintetizar apenas um dos parâmetros do hemograma.
Outras simplificações devem ser devidamente justificadas.
Deverá ser entregue um notebook Google Colab, explicitando e explicando sua implementação.

Responda à seguinte pergunta: como você garante que a sua síntese de dados é realista?
Explicite os critérios utilizados.

Sugere-se que o material suplementar anexo seja estudado, mas adianta-se que o mesmo não contêm a solução do problema.
Servem apenas como uma introdução a aspectos relevantes a serem considerados na síntese de sinais e um exemplo de abordagem na saúde.

# Ingestão Dados